Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd

#df = pd.read_csv('/datasets/toxic_comments.csv')
df = pd.read_csv('/Users/ag/Documents/ML/Project/toxic_comments.csv')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


- лемматизировать текст будем с помощью TextBlob
- для Wordnet будет использовать POS-теги, начинающиеся на: J, N, V, R. полный список ниже.

        'CC':None, # coordin. conjunction (and, but, or)  
        'CD':wn.NOUN, # cardinal number (one, two)             
        'DT':None, # determiner (a, the)                    
        'EX':wn.ADV, # existential ‘there’ (there)           
        'FW':None, # foreign word (mea culpa)             
        'IN':wn.ADV, # preposition/sub-conj (of, in, by)   
        'JJ':[wn.ADJ, wn.ADJ_SAT], # adjective (yellow)                  
        'JJR':[wn.ADJ, wn.ADJ_SAT], # adj., comparative (bigger)          
        'JJS':[wn.ADJ, wn.ADJ_SAT], # adj., superlative (wildest)           
        'LS':None, # list item marker (1, 2, One)          
        'MD':None, # modal (can, should)                    
        'NN':wn.NOUN, # noun, sing. or mass (llama)          
        'NNS':wn.NOUN, # noun, plural (llamas)                  
        'NNP':wn.NOUN, # proper noun, sing. (IBM)              
        'NNPS':wn.NOUN, # proper noun, plural (Carolinas)
        'PDT':[wn.ADJ, wn.ADJ_SAT], # predeterminer (all, both)            
        'POS':None, # possessive ending (’s )               
        'PRP':None, # personal pronoun (I, you, he)     
        'PRP$':None, # possessive pronoun (your, one’s)    
        'RB':wn.ADV, # adverb (quickly, never)            
        'RBR':wn.ADV, # adverb, comparative (faster)        
        'RBS':wn.ADV, # adverb, superlative (fastest)     
        'RP':[wn.ADJ, wn.ADJ_SAT], # particle (up, off)
        'SYM':None, # symbol (+,%, &)
        'TO':None, # “to” (to)
        'UH':None, # interjection (ah, oops)
        'VB':wn.VERB, # verb base form (eat)
        'VBD':wn.VERB, # verb past tense (ate)
        'VBG':wn.VERB, # verb gerund (eating)
        'VBN':wn.VERB, # verb past participle (eaten)
        'VBP':wn.VERB, # verb non-3sg pres (eat)
        'VBZ':wn.VERB, # verb 3sg pres (eats)
        'WDT':None, # wh-determiner (which, that)
        'WP':None, # wh-pronoun (what, who)
        'WP$':None, # possessive (wh- whose)
        'WRB':None, # wh-adverb (how, where)
        '$':None, #  dollar sign ($)
        '#':None, # pound sign (#)
        '“':None, # left quote (‘ or “)
        '”':None, # right quote (’ or ”)
        '(':None, # left parenthesis ([, (, {, <)
        ')':None, # right parenthesis (], ), }, >)
        ',':None, # comma (,)
        '.':None, # sentence-final punc (. ! ?)
        ':':None # mid-sentence punc (: ; ... – -)

In [53]:
%pip install textblob
from textblob import TextBlob, Word

Note: you may need to restart the kernel to use updated packages.


In [54]:
import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

In [55]:
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

- кроме POS-тегов не будем проводить никаких преобразований текстов, особенно регулярки
- исходим из того, что цифры мы тоже оставляем, поскольку есть огромное количество устоявшихся оскорбительных мемов, основанных на использовании цифр, а также литспик 1337
- проверяем как это работает на первой записи в датасете
- также не забываем, что дальше мы будем работать в нижнем регистре

In [56]:
corpus = df['text'].str.lower()

In [57]:
print("Исходный текст:", corpus[0])
print("Лемматизированный текст:", lemmatize_with_postag(corpus[0]))

Исходный текст: explanation
why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now.89.205.38.27
Лемматизированный текст: explanation why the edits make under my username hardcore metallica fan be revert they be n't vandalisms just closure on some gas after i vote at new york doll fac and please do n't remove the template from the talk page since i 'm retired now.89.205.38.27


- имеем не страшные 'm и n't, вполне уместно с ними ничего не делать и индексировать.
- также есть ip адрес, ну пусть будет, вероятно и днс-имена в тексте встречаются. лучше и их проиндексировать, насколько токсична ссылка на порносайт - вопрос риторический.
- то же самое и с эмейлами
- в общем, еще раз убеждаемся, что дополнительно ничего делать не будем

In [58]:
%%time
df['text_lemm'] = df['text'].str.lower().apply(lemmatize_with_postag)

CPU times: user 13min 12s, sys: 20.2 s, total: 13min 32s
Wall time: 13min 44s


- чтобы кернел не болел - сохраняем результаты лемматизации в файл, и закомментируем лемматизацию

In [59]:
df.to_csv('text_lemm.csv', index=False)

- а потом результат опять загрузим из файла

In [60]:
df_text = pd.read_csv('text_lemm.csv')
df_text = df_text[['text_lemm', 'toxic']]
df_text.head()

,text_lemm,toxic
0,explanation why the edits make under my userna...,0
1,d'aww he match this background colour i 'm see...,0
2,hey man i 'm really not try to edit war it 's ...,0
3,more i ca n't make any real suggestion on impr...,0
4,you sir be my hero any chance you remember wha...,0


# 2. Обучение

- делим датасет на выборки, делаем таргеты

In [61]:
from sklearn.model_selection import train_test_split

In [62]:
train, test = train_test_split(df_text, test_size=0.3, random_state=12345)

In [63]:
train = train.dropna()
test = test.dropna()

In [64]:
train.reset_index(drop=True)

,text_lemm,toxic
0,i think access deny be a perfectly valid usern...,0
1,please stop abuse the word hopefully ignorant ...,1
2,progressive dance music i see you do n't surre...,0
3,japanese character they annoy me this be an en...,0
4,yeah it do seem kind of over and just to let y...,0
...,...,...
111694,gam keep your crap off my talk page your kkk s...,0
111695,i correct what you listed.-,0
111696,hmm yes a i say i also watch the emily ruete-a...,0
111697,kinda like miachael jackson,0


In [65]:
test.reset_index(drop=True)

,text_lemm,toxic
0,ahh shut the fuck up you douchebag sand nigger...,1
1,reply there be no such thing a texas commerce ...,0
2,reply hey you could at least mention jasenovac...,0
3,thats fine there be no deadline chi,0
4,dyk nomination of mustarabim hello your submis...,0
...,...,...
47867,because that person watch wwe smackdown and sa...,0
47868,his tour end ten year ago both elvis and mj ha...,0
47869,people can judge for themselves what you inten...,0
47870,welcome hello kronoss13 and welcome to wikiped...,0


In [66]:
target_train = train[['toxic']]
target_test = test[['toxic']]

- далее лемматизированный текст преобразуем в вектор и считаем TF IDF

In [67]:
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [68]:
%%time
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
corpus_train = train['text_lemm'].values.astype('U')
tf_idf_train = count_tf_idf.fit_transform(corpus_train)

[nltk_data] Downloading package stopwords to /Users/ag/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


CPU times: user 8.68 s, sys: 1.08 s, total: 9.76 s
Wall time: 10.3 s


In [69]:
%%time
corpus_test = test['text_lemm'].values.astype('U')
tf_idf_test = count_tf_idf.transform(corpus_test)

CPU times: user 3.53 s, sys: 347 ms, total: 3.88 s
Wall time: 3.95 s


-----

### Логистическая регрессия

- обучение начинаем с самого простого

In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [74]:
import warnings
warnings.filterwarnings("ignore")

In [75]:
target_train['toxic'].value_counts()

0    100351
1     11348
Name: toxic, dtype: int64

- дисбаланс классов определенно есть, но не такой чтобы расходовать ресурсы и время на преобразования выборок
- попробуем ограничиться балансом классов

In [76]:
%%time
model_lg = LogisticRegression(class_weight={0:0.3,1:0.7})
model_lg.fit(tf_idf_train, target_train)

CPU times: user 11.5 s, sys: 165 ms, total: 11.6 s
Wall time: 3.12 s


LogisticRegression(C=1.0, class_weight={0: 0.3, 1: 0.7}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [77]:
%%time
print("f1 score: {:.2f}".format(f1_score(target_test, model_lg.predict(tf_idf_test))))

f1 score: 0.76
CPU times: user 18.3 ms, sys: 8.46 ms, total: 26.7 ms
Wall time: 42.4 ms


- целевой F1 превзошли, можно заканчивать, на подбор и обучение модели потрачено менее 5 секунд

--------

### Случайный лес

- остальные модели попробуем подобрать с помощью hyperopt
- лимиты на время обучения делаем щадащими, тратить время и ресурсы уже ни к чему

In [78]:
%pip install hyperopt

Note: you may need to restart the kernel to use updated packages.


In [79]:
import numpy as np
import hyperopt as hp
from hyperopt import Trials,fmin,STATUS_OK

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer

- поскольку hyperopt умеет только минимизировать лосс-функцию, то пишем кастомный скорер, благодаря которому мы будем минимизировать отрицательный f1, то есть --> максимизировать положительный

In [80]:
def f1_neg(truth, predictions):
    return - f1_score(truth, predictions.round())

f1_scorer = make_scorer(f1_neg, greater_is_better=True, needs_proba=True)

In [82]:
def objective(params):
    params = {'n_estimators': int(params['n_estimators']), 'max_depth': int(params['max_depth'])}
    clf = RandomForestClassifier(n_jobs=4, class_weight='balanced', **params)
    score = cross_val_score(clf, tf_idf_train, target_train.values.ravel(), scoring=f1_scorer, cv=StratifiedKFold()).mean()
    print("f1 score: {:.3f}, params: {}".format(score, params))
    return score

space = {
    'n_estimators': hp.hp.choice('n_estimators',  np.arange(100, 500, 100, dtype=int)),
    'max_depth': hp.hp.choice('max_depth', np.arange(10, 30, 10, dtype=int))
}

best_rfc = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=3)

f1 score: -0.366, params: {'n_estimators': 100, 'max_depth': 10}
f1 score: -0.404, params: {'n_estimators': 400, 'max_depth': 20}                
f1 score: -0.400, params: {'n_estimators': 300, 'max_depth': 20}                 
100%|██████████| 3/3 [12:06<00:00, 242.10s/trial, best loss: -0.4044811449838203]


In [95]:
print("Hyperopt: {}".format(best_rfc))

Hyperopt: {'max_depth': 1, 'n_estimators': 3}


In [87]:
%%time
model_rfc = RandomForestClassifier(n_jobs=4, class_weight='balanced', **{'max_depth': 20, 'n_estimators': 400})
model_rfc.fit(tf_idf_train, target_train.values.ravel())
print("f1 score: {:.2f}".format(f1_score(target_test, model_rfc.predict(tf_idf_test))))

f1 score: 0.41
CPU times: user 1min 12s, sys: 912 ms, total: 1min 13s
Wall time: 19.2 s


- F1 значительно хуже логистической регрессии, результат улучшить можно, потратив больше времени, но мы этого делать не будем

------

### LightGBM

- бустинг расчехляем в качестве факультатива
- естественно тратить на подбор оптимальных параметров не будем слишком много

In [88]:
%pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [89]:
import lightgbm as lgb
from lightgbm import Dataset

In [90]:
def objective(params):
    params = {
        'num_leaves': int(params['num_leaves']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
    }
    
    clf = lgb.LGBMClassifier(
        n_estimators=500,
        learning_rate=0.01,
        **params
    )
    
    score = cross_val_score(clf, tf_idf_train, target_train.values.ravel(), scoring=f1_scorer, cv=StratifiedKFold()).mean()
    print("f1 score: {:.3f}, params: {}".format(score, params))
    return score

space = {
    'num_leaves': hp.hp.choice('num_leaves', np.arange(40, 120, 20, dtype=int)),
    'colsample_bytree': hp.hp.quniform('colsample_bytree', 0.5, 1.0, 0.1),
}

best_lgb = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=3)

f1 score: -0.720, params: {'num_leaves': 40, 'colsample_bytree': '0.700'}
f1 score: -0.745, params: {'num_leaves': 80, 'colsample_bytree': '0.900'}       
f1 score: -0.719, params: {'num_leaves': 40, 'colsample_bytree': '0.900'}        
100%|██████████| 3/3 [31:59<00:00, 639.86s/trial, best loss: -0.7451936773384316]


In [91]:
print("Hyperopt: {}".format(best_lgb))

Hyperopt: {'colsample_bytree': 0.9, 'num_leaves': 2}


In [92]:
d_train_lgb = lgb.Dataset(tf_idf_train,target_train)

In [93]:
%%time
model_lgb = lgb.LGBMClassifier(n_estimators=500, learning_rate=0.01, **{'num_leaves': 80, 'colsample_bytree': '0.900'})
model_lgb.fit(tf_idf_train, target_train.values.ravel())

CPU times: user 26min 8s, sys: 25 s, total: 26min 33s
Wall time: 3min 52s


LGBMClassifier(boosting_type='gbdt', class_weight=None,
               colsample_bytree='0.900', importance_type='split',
               learning_rate=0.01, max_depth=-1, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=500,
               n_jobs=-1, num_leaves=80, objective=None, random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [94]:
%%time
print("f1 score: {:.2f}".format(f1_score(target_test, model_lgb.predict(tf_idf_test))))

f1 score: 0.75
CPU times: user 42.6 s, sys: 220 ms, total: 42.9 s
Wall time: 6.02 s


- F1 тоже пройден, но это было все равно что вызывать вертолет вместо поездки на самокате.

# 3. Выводы

1. Целевой F1 пройден на логистической регрессии с подготовкой корпуса при помощи TF IDF, лемматизацией с POS-токенами, но без регулярных выражений и прочих преобразований.
2. Более сложные модели и больше времени на подбор параметров дадут улучшенный результат.
3. Подобные учебные проекты вероятно надо сопровождать учебными же материалами по оптимизации кода и ресурсов в рамках курса.

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны